# Import libs and declare variables

In [33]:
import pandas as pd

input_file_name = "../dados/base_desafio_cartola.csv"

df = pd.read_csv(input_file_name)

df.head(3)

,user,sexo,uf,idade,dias,pviews,visitas,tempo_total,device,futebol,...,natacao,basquete,handebol,volei,tenis,canoagem,saltos_ornamentais,home,home_olimpiadas,cartola_status
0,1912357,NaN,NaN,NaN,12,31,20,3868.477,pc_e_m,0.000,...,0.0,0.0,0.000,46.851,0.0,0.000,0.000,0.0,606.427,Não Cartola
1,1914630,NaN,NaN,NaN,2,8,5,427.352,pc_only,201.338,...,0.0,0.0,0.000,45.000,0.0,0.000,0.000,0.0,0.000,Não Cartola
2,1915241,NaN,NaN,NaN,14,44,23,14758.755,pc_e_m,4867.997,...,90.0,0.0,2471.093,300.573,0.0,1956.698,431.616,0.0,91.721,Não Cartola


## Cria variável alvo binária

In [34]:
label = "cartola_status"
label_pro = "is_pro"

df = df[df[label].notna()]
df[label_pro] = (df[label] == 'Cartola Pro').astype(int)

## Geração de novas variáveis

In [35]:
import feature_generation

df_feat = feature_generation.get_new_features(df)

In [113]:
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error  # Importa a função para calcular o erro absoluto percentual médio (MAPE)
from sklearn.metrics import mean_absolute_error  # Importa a função para calcular o erro absoluto médio (MAE)
from sklearn.metrics import mean_squared_error  # Importa a função para calcular o erro quadrático médio (MSE)

from sklearn.metrics import accuracy_score  # Importa a função para calcular a acurácia
from sklearn.metrics import f1_score  # Importa a função para calcular o F1-Score


def print_metrics(y_real: np.array = [],
                  y_pred: np.array = [],
                  method: str = ""):
    """
    Função para imprimir métricas de avaliação de modelos de regressão ou classificação.

    Parâmetros:
    y_real (array-like): Valores reais.
    y_pred (array-like): Valores previstos pelo modelo.
    method (str): Tipo de modelo - "Regressor" para modelos de regressão, qualquer outro valor para modelos de classificação.
    """
    if method == "Regressor":
        # Calcula e imprime as métricas para modelos de regressão
        print(f"MAPE: {round(mean_absolute_percentage_error(y_real, y_pred)*100, 1)}%")
        print(f"MAE: {round(mean_absolute_error(y_real, y_pred), 2)}")
        print(f"MSE: {round(mean_squared_error(y_real, y_pred), 2)}\n")
    else:
        # Calcula e imprime as métricas para modelos de classificação
        print(f"Accuracy: {round(accuracy_score(y_real, y_pred)*100, 2)}%")
        print(f"F1-Score: {round(f1_score(y_real, y_pred, average='macro'), 2)}\n")

In [114]:
from sklearn.model_selection import train_test_split  # Importa a função para dividir os dados em conjuntos de treino e teste
from sklearn.ensemble import RandomForestRegressor  # Importa o modelo de regressão RandomForest
from sklearn.ensemble import RandomForestClassifier  # Importa o modelo de classificação RandomForest

def get_model(method: str = ""):
    """
    Função para obter o modelo adequado com base no tipo de problema.

    Parâmetros:
    method (str): Tipo de modelo - "Regressor" para problemas de regressão, qualquer outro valor para problemas de classificação.

    Retorna:
    Modelo RandomForestRegressor ou RandomForestClassifier.
    """
    if method == "Regressor":
        return RandomForestRegressor()
    return RandomForestClassifier()

# Loop para avaliar diferentes características e tipos de modelos
for (feature_to_be_filled, method_type) in [("sexo", "Classifier"),
                                            ("uf", "Classifier"),
                                            ("idade", "Regressor")]:
    # Remove linhas com valores faltantes
    df_complete = df_feat.dropna()
    
    # Seleciona as colunas numéricas, exceto a coluna alvo (feature_to_be_filled)
    X_train = df_complete.drop(columns=[feature_to_be_filled]).select_dtypes(np.number)
    y_train = df_complete[feature_to_be_filled]

    # Divide os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,
                                                        test_size=0.3,
                                                        random_state=42)

    # Obtém o modelo apropriado com base no tipo de problema (regressor ou classificador)
    model = get_model(method_type)
    
    # Treina o modelo com os dados de treino
    model.fit(X_train, y_train)
    
    # Faz previsões no conjunto de teste
    y_pred = model.predict(X_test)

    # Imprime a feature avaliada e as métricas de desempenho
    print(f"Feature evaluated: {feature_to_be_filled}")
    print_metrics(y_test, y_pred, method_type)

Feature evaluated: sexo
Accuracy: 95.42%
F1-Score: 0.49

Feature evaluated: uf
Accuracy: 29.6%
F1-Score: 0.03

Feature evaluated: idade
MAPE: 32.6%
MAE: 7.68
MSE: 4604.56

